## This notebook is for Segmenting and Clustering Neighborhoods in Toronto Assignment





# Part 1

In [2]:
#Import Pandas and Numpy
import pandas as pd
import numpy as np

Use pandas 'read_html' to read table from URL



In [3]:
#Read in data from HTML
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

dfs = pd.read_html(url)

The above produced a list, looking at the list showed that the information I need was at index 0




In [4]:
#I reset the dfs variable so it contains only the information I need, which is at index 0
dfs = dfs[0]

Checking that our data is right



In [5]:
dfs

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


I'm dropping all rows that have 'Not assigned' as a Borough

In [6]:
#Drop unwanted rows
df = dfs[~dfs['Borough'].isin(['Not assigned'])]

#check our df
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Let's print how many rows we have in our dataframe.


In [7]:
#print rows in dataframe using shape
print('We have {} rows in our dataframe'.format(df.shape[0]))

We have 103 rows in our dataframe


# Part 2

We'll use geocoder package to get latitude and longitude coordinates for each neighborhood and insert that into our dataframe.

In [8]:
# Import geocoder
import geocoder
print('Geocoder imported')

Geocoder imported


Below I'll use a for loop to fill my dataframe with latlng for neighborhoods from geocoder

In [9]:


for postal_code in df['Postal Code']:
    lat_lng_coords = None
    
    while lat_lng_coords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code), key='**REMOVED**')
        lat_lng_coords = g.latlng
        df.loc[df['Postal Code'] == postal_code, 'latitude'] = lat_lng_coords[0]
        df.loc[df['Postal Code'] == postal_code, 'longitude'] = lat_lng_coords[1]
    


C:\Users\jyana\Documents\Coursera\Coursera_Capstone\venv\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jyana\Documents\Coursera\Coursera_Capstone\venv\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Now I'll take a look at my new data:


In [10]:
df


,Postal Code,Borough,Neighborhood,latitude,longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3

Get toronto neighborhoods from data in part 2

In [11]:
toronto_df = df[df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M4E,East Toronto,The Beaches,43.676357,-79.293031


Use geopy to get lat/long values for Toronto

In [12]:
import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='cap_project')
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}'.format(latitude, longitude))


The coordinates of Toronto are 43.6534817, -79.3839347


Create a map of Toronto


In [13]:
# Import relevant Libraries
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


In [14]:
# create map using lat/long values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['latitude'], toronto_df['longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False
    ).add_to(toronto_map)

toronto_map

Define Foursquare API values


In [15]:
CLIENT_ID = '**REMOVED**'
CLIENT_SECRET = '**REMOVED**'
VERSION = '20180605'
limit = 100
radium = 500

Borrowed function from Lab

In [16]:
import requests
import json
import pandas.io.json


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

Apply Function to our data

In [17]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'], latitudes=toronto_df['latitude'], longitudes=toronto_df['longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

Check the shape of the resulting dataframe



In [18]:
print('The shape of the dataframe is {}. The dataset has {} rows.'.format(toronto_venues.shape, toronto_venues.shape[0]))
toronto_venues.head()

The shape of the dataframe is (1606, 7). The dataset has 1606 rows.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Check how many venues were returned for each neighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


Analyze the Neighborhoods

Create dummy variables based on venue category for each row in dataframe



In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood']=toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# check the number of rows. It must be equal to that of toronto_venues
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (toronto_onehot.shape,toronto_onehot.shape[0]))

# print top 5 rows
toronto_onehot.head()

The shape of the dataframe is (1606, 233). The dataset has 1606 rows.


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

# Check shape of toronto_grouped
print('The shape of the dataframe is {}. The dataset has {} rows.'.format(
    toronto_grouped.shape, toronto_grouped.shape[0]
))

# Lets look at the head
toronto_grouped.head()

The shape of the dataframe is (39, 233). The dataset has 39 rows.


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Business reply mail Processing Centre,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.015873,0.0,0.0,0.0,0.0,0.0,0.015873


Clustering:

Initiate Clustering

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels for each row

kmeans.labels_



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0,
       0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0])

Add cluster labels to toronto_grouped and then join back to toronto_df

In [23]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

# left join
toronto_merged = pd.merge(toronto_df, toronto_grouped, how='left', left_on='Neighborhood', right_on='Neighbourhood')

# delete duplicate column
toronto_merged.drop(['Neighbourhood'], axis=1, inplace=True)

# Rename Column

toronto_merged.rename(columns={'Neighborhood_x': 'Neighborhood'}, inplace=True)

toronto_merged.head()

,Postal Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.022222
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.029412,0.0,0.029412
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.01,0.01,0.000000,0.000000,0.0,0.000000
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.013158,0.00,0.00,0.013158,0.000000,0.0,0.000000
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000


In [24]:
# Check the shape of our new data
toronto_merged.shape

(39, 238)

Visualize the resulting clusters



In [26]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
marker_colors = []
for lat, lon, poi, cluster in zip(
    toronto_merged['latitude'],
    toronto_merged['longitude'],
    toronto_merged['Neighborhood'],
    toronto_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius = 5,
            popup = label,
            color = rainbow[cluster-1],
            fill = True,
            fill_color = rainbow[cluster-1],
            fill_opacity = 0.7
        ).add_to(map_clusters)
        
map_clusters

